# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****tM0A
LANGSMITH_API_KEY=****bc46
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [3]:
from langchain.agents import create_agent

In [4]:
agent = create_agent(
    model="openai:gpt-5",
    system_prompt="You are a full-stack comedian",
)

## No Steaming (invoke)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Why do programmers prefer dark mode? Because light attracts bugs.

Want another—tech, dad, or wordplay?


## values
You have seen this streaming mode in our examples so far. 

In [6]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

I’m reading a book on anti-gravity—it's impossible to put down.


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [7]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

In our bright and bouncy house, the day begins with cheer,
Toast pops up like parachutes and jelly lands near.

The milk does tiny cartwheels, the cereal shouts “Hooray!”
A sock plays hide-and-seek and wins by running away.

The dog’s our doorbell manager, with wags for every guest,
The cat audits all laps and chooses only the best.

Our chore chart’s a treasure map with Gold Star Island’s glow,
We battle Mount Laundry and the Dust-Bunny snow.

At homework time, the Wi‑Fi is a dragon we befriend—
We offer snacks of patience till the loading bars will bend.

We measure out our laughter like sprinkles on a cake,
And pass around the giggles when a pun needs a break.

At dinner, tiny trees of broccoli grow tall,
We are giants in the forest taking teeny bites of all.

The dishes start a drumline in the sink’s tin band,
We clap with soapy mittens like a squeaky washing hand.

Then toothbrushes perform a minty lullaby,
And pajamas parade under moonlight’s tie-dye.

Night-lights turn to lighth

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [8]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='2925fabf-3c7e-4a14-9a72-531240436fbf')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='2925fabf-3c7e-4a14-9a72-531240436fbf'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 132, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ca9Mw5d8Cqw2CIFh7xs3xeT7LnCbe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--390b0a79-531d-4131-8cba-c548963eab78-0', tool_calls=[{'name': 'get_we

In [9]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco, CA')
('custom', 'Acquired data for city: San Francisco, CA')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [18]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values"],
):
    if chunk[0] == "values":
        print(chunk[1])

{'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='79d6ff20-a97e-4e67-9bd5-40790821c14c')]}
{'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='79d6ff20-a97e-4e67-9bd5-40790821c14c'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 154, 'prompt_tokens': 132, 'total_tokens': 286, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ca9VOH3NSyLpf5DsiMxZXsAwxycpk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9bca45db-9da8-4c5d-a70d-cbd93a1122f8-0', tool_calls=[{'name': 'get_weather', 'args': {'cit